In [1]:
import numpy as np
import xgboost
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook
import sklearn.preprocessing
import gc

In [2]:
# dataframe display 옵션
pd.set_option('display.max_columns' , 30)
pd.set_option('display.width', None)

In [3]:
data = pd.read_csv('./data/train_V2.csv')

In [ ]:
train_df, sample_df = train_test_split(data, train_size = 0.95)

In [ ]:
## 메모리 절약
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    #start_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    #end_mem = df.memory_usage().sum() / 1024**2
    #print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    #print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
sample_df = reduce_mem_usage(sample_df)

In [ ]:
del data
del train_df
gc.collect()

In [ ]:
# 회귀분석을 하기 위해 문자열을 범주값으로 변환
matchTyp = {'squad-fpp': 0, 'duo': 1, 'solo-fpp': 2, 'squad': 3, 'duo-fpp': 4, 'solo': 5,
       'normal-squad-fpp': 6, 'crashfpp': 7, 'flaretpp': 8, 'normal-solo-fpp': 9,
       'flarefpp': 10, 'normal-duo-fpp': 11, 'normal-duo': 12, 'normal-squad': 13,
       'crashtpp': 14, 'normal-solo': 15 }

sample_df['matchType'] = sample_df['matchType'].replace(matchTyp)

In [ ]:
sample_df.dropna(inplace = True)
print(sample_df.isnull().any().any())

In [ ]:
sample_df.columns

In [ ]:
del sample_df['Id']
del sample_df['groupId']
del sample_df['matchId']

In [ ]:
len(sample_df.columns)

In [ ]:
sample_np = np.array(sample_df)

In [ ]:
X = sklearn.preprocessing.minmax_scale(sample_np[:,0:25])
Y = sample_np[:,25]

In [ ]:
# 단위 변환 - minmaxscale

In [ ]:
# fit model
model = xgboost.XGBClassifier()
model.fit(X,Y)

In [ ]:
# plot feature importance
xgboost.plot_importance(model)
pyplot.show()